## DreamBooth on Stable Diffusion 2.1

DreamBooth is a powerful technique for personalizing latent diffusion models, such as Stable Diffusion, allowing the model to be fine-tuned to generate specific images based on a limited set of data.

### Step 0: Environment configuration

This command installs the "diffusers" library directly from the Hugging Face GitHub repository.The diffusers library is used to work with latent diffusion models, such as Stable Diffusion, and provides tools for image generation, fine-tuning, and other functionalities.


In [ ]:
!pip install -r ../requirements.txt


### Configuration and Secrets Loading
In this section, we load configuration parameters and API keys from separate YAML files. This separation helps maintain security by keeping sensitive information (API keys) separate from configuration settings.

- **config.yaml**: Contains non-sensitive configuration parameters like model sources and URLs
- **secrets.yaml**: Contains sensitive API keys for services like Galileo and HuggingFace


In [ ]:
import os
import sys

# Add the src directory to the path to import utils
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))
from src.utils import load_config_and_secrets

config_path = "../../configs/config.yaml"
secrets_path = "../../configs/secrets.yaml"

config, secrets = load_config_and_secrets(config_path, secrets_path)

In [ ]:
from core.custom_metrics.image_metrics_scorers import entropy_scorer, complexity_scorer, set_custom_image_path
from core.deploy.deploy_image_generation import deploy_model

from huggingface_hub import snapshot_download
import promptquality as pq
import glob

### Download model local

This code imports the snapshot_download function from the huggingface_hub library to download the latest version of the "stabilityai/stable-diffusion-2-1" model. It sets a local directory for saving the model (local_model_path), and the download is configured to be resumable in case it is interrupted, with an etag timeout set to 60 seconds.

In [ ]:

# Download the snapshot directly to the local directory
local_model_path = os.path.join("..", "..", "..", "local", "stable-diffusion-2-1")

# Downloading the latest revision of the "stabilityai/stable-diffusion-2-1" model
snapshot_download(
    repo_id="stabilityai/stable-diffusion-2-1", 
    local_dir=local_model_path,
    resume_download=True,
    etag_timeout=60  
)

### Step 1: Load the Model
We load the pre-trained Stable Diffusion 2.1 model from Hugging Face and move it to the GPU for efficient execution

In [ ]:
from core.local_inference.inference import StableDiffusionPipelineOutput, load_config, run_inference

config = load_config()

run_inference(
    prompt="A beautiful landscape",
    height=768,
    width=768,
    num_images=1,
    num_inference_steps=60
)


## Step 3: Training Dreambooth

This Bash script checks the available GPUs using PyTorch, selects a multi-GPU or single-GPU configuration file accordingly, and then launches a training script (using accelerate) for Dreambooth on Stable Diffusion with specified parameters. It also records and calculates the training duration.

In [ ]:
%%bash
NUM_GPUS=$(python3 -c "import torch; print(torch.cuda.device_count())")

if [ "$NUM_GPUS" -ge 2 ]; then
  CONFIG_FILE="../data/config/default_config_multi-gpu.yaml"
  echo "Detected $NUM_GPUS GPUs, using $CONFIG_FILE"
else
  CONFIG_FILE="../data/config/default_config_one-gpu.yaml"
  echo "Detected $NUM_GPUS GPU, using $CONFIG_FILE"
fi

START=$(date +%s)

accelerate launch --config_file $CONFIG_FILE core/train/train_dreambooth_aistudio.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1"  \
  --instance_data_dir="../data/img" \
  --output_dir="./dreambooth/" \
  --instance_prompt="A modern laptop on a sandy beach with the ocean in the background, sunlight reflecting off the screen" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=400 \
  --logging_dir="/phoenix/tensorboard/tensorlogs" \
  --report_to="tensorboard" \
  --validation_prompt="A photo of an HP laptop on the sand with a sunset over the ocean in the background." \
  --num_validation_images=1 \
  --validation_steps=100

END=$(date +%s)
DIFF=$(( $END - $START ))


## Inference Local Model

This code imports functions from the inference_dreambooth module, loads a configuration, and then runs inference to generate images. It uses a prompt to create three images with a resolution of 768x768 pixels, executing 100 inference steps per image.

In [ ]:
from core.dreambooth_inference.inference_dreambooth import StableDiffusionPipelineOutput, load_config, run_inference

config = load_config()

run_inference(
    prompt="A high-quality photo of an HP laptop placed on the sand at the beach, with a sunset over the ocean in the background.", 
    height=768, 
    width=768, 
    num_images=3, 
    num_inference_steps=100  
)


## Galileo Evaluate Custom metrics
Galileo GenAI Studio supports Custom Metrics (programmatic or GPT-based) for all your Evaluate and Observe projects. 

In [ ]:

#########################################
# In order to connect to Galileo, create a secrets.yaml file in the same folder as this notebook
# This file should be an entry called Galileo, with the your personal Galileo API Key
# Galileo API keys can be created on https://console.hp.galileocloud.io/settings/api-keys
#########################################

with open('secrets.yaml') as file:
    secrets = yaml.safe_load(file)
    os.environ['GALILEO_API_KEY'] = secrets["Galileo"]

os.environ['GALILEO_CONSOLE_URL'] = "https://console.hp.galileocloud.io/" 

pq.login(os.environ['GALILEO_CONSOLE_URL'])

In [ ]:

#########################################

# Returns the path of the most recent image that matches the specified pattern.

#########################################

def get_latest_generated_image(directory: str = "./", prefix: str = "local_model_result_", ext: str = ".png") -> str:
    files = glob.glob(os.path.join(directory, f"{prefix}*{ext}"))
    if not files:
        raise FileNotFoundError("No generated images founda.")
    latest_file = max(files, key=os.path.getmtime)
    return latest_file

In [ ]:

config = load_config()

prompt_text = ("A high-quality photo of an HP laptop placed on the sand at the beach, "
               "with a sunset over the ocean in the background.")

run_inference(
    prompt=prompt_text, 
    height=768, 
    width=768, 
    num_images=1, 
    num_inference_steps=100  
)

generated_image_path = get_latest_generated_image()

set_custom_image_path(generated_image_path)

template = prompt_text

result_custom = pq.run(template=template, scorers=[entropy_scorer, complexity_scorer])
print("Result:", result_custom)

## Model Service

Using MLflow, we will save and load the model in an integrated manner, enabling the traceability and reproducibility of experiments. MLflow will facilitate model versioning, monitoring, and deployment, ensuring a robust pipeline for your project.

In [ ]:
deploy_model()